In [68]:
import sys,os
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, precision_score, recall_score,roc_auc_score,accuracy_score
from sklearn import metrics
from sklearn.feature_extraction.text import HashingVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize 
from keras.layers import Input, Concatenate
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPool2D, Reshape
from keras.optimizers import Adam, SGD, RMSprop

In [66]:
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs
from sklearn.preprocessing import LabelBinarizer

In [8]:
stop_words = set(stopwords.words('english')) 


In [6]:
def remove_hashtags(tweet):
    result = ''

    for word in tweet.split():
        if word.startswith('#') or word.startswith('@'):
            result += word[1:]
            result += ' '
        else:
            result += word
            result += ' '

    return result

In [27]:
# stemmer = SnowballStemmer('english')
# stem_map={}

# stopW = stopwords.words('english')
# emoji_pattern = re.compile("["
#      u"\U0001F600-\U0001F64F"  
#      u"\U0001F300-\U0001F5FF"  
#      u"\U0001F680-\U0001F6FF"  
#      u"\U0001F1E0-\U0001F1FF"  
#      u"\U00002702-\U000027B0"
#      u"\U000024C2-\U0001F251"
#      "]+", flags=re.UNICODE)

# def load_data(filename):
#     n = ['id', 'text','HS','TR','AG']
#     given_data = pd.read_csv(filename, sep='\t',error_bad_lines=False, names=n, usecols=['text','HS','TR','AG'], skiprows=1)
#     raw_data = given_data['text'].values
#     labels_TR = list(map(int,given_data['TR'].values))
#     labels_AG = list(map(int,given_data['AG'].values))
#     labels_HS = list(map(int,given_data['HS'].values))
#     return raw_data,labels_TR,labels_AG,labels_HS


# def preprocess(tweet,pre_process = True,remove_URL = True, remove_mentions = True, remove_emojis = True, token_method = True, stem = True,remove_hashtag = True):

#     if remove_URL:
#         tweet = re.sub('http\\S+', '', tweet, flags=re.MULTILINE)
    
#     if remove_mentions:
#         tweet = re.sub('@[^\s]+','USER', tweet)
    
#     if pre_process:
#         tweet = re.sub('@(\\w{1,15})\b', '', tweet)
#         tweet = tweet.replace("via ", "")
#         tweet = tweet.replace("RT ", "")
#         tweet = tweet.replace("ё", "е")
#         tweet = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', tweet)
#         tweet = re.sub(' +',' ', tweet)
#         tweet = tweet.lower()
    
#     if remove_emojis:    
#         tweet = emoji_pattern.sub(r'', tweet)
    
#     if remove_hashtag:
#         tweet = remove_hashtags(tweet)
#     stemmed_text_token=[]
    
#     tokens = []
#     if token_method:
#         twtk = TweetTokenizer(reduce_len=True)
#         tokens = twtk.tokenize(tweet)
#     else:
#         tokens = nltk.tokenize(tweet)
        
#     text_token = []
#     for token in tokens:
#         if token not in stop_words:
#             text_token.append(token)
#     for token in text_token:
#         if token=='':
#             continue
#         elif token=='USER' or token == 'URL': 
#             stemmed_text_token.append(token)
#         else:
#             a=stem_map.get(token,0)
#             if a==0:
#                 a=stemmer.stem(token)
#                 stem_map[token]=a
#             if stem:
#                 stemmed_text_token.append(a)
#             else:
#                 stemmed_text_token.append(token)
                
#     return ' '.join(stemmed_text_token)


In [188]:
stemmer = SnowballStemmer('english')
stem_map={}

stopW = stopwords.words('english')
emoji_pattern = re.compile("["
     u"\U0001F600-\U0001F64F"  
     u"\U0001F300-\U0001F5FF"  
     u"\U0001F680-\U0001F6FF"  
     u"\U0001F1E0-\U0001F1FF"  
     u"\U00002702-\U000027B0"
     u"\U000024C2-\U0001F251"
     "]+", flags=re.UNICODE)

def load_data(filename):
    n = ['id', 'text','HS','TR','AG']
    given_data = pd.read_csv(filename, sep='\t',error_bad_lines=False, names=n, usecols=['text','HS','TR','AG'], skiprows=1)
    raw_data = given_data['text'].values
    labels_TR = list(map(int,given_data['TR'].values))
    labels_AG = list(map(int,given_data['AG'].values))
    labels_HS = list(map(int,given_data['HS'].values))
    return raw_data,labels_TR,labels_AG,labels_HS

def preprocess(tweet):
    # ' '.join([word for word in tweet.spilt() ])
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', tweet)
    tweet = re.sub('@[^\s]+','USER', tweet)
    tweet = tweet.replace("ё", "е")
    tweet = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', tweet)
    tweet = re.sub(' +',' ', tweet)
    tweet = emoji_pattern.sub(r'', tweet)

    stemmed_text_token=[]
    twtk = TweetTokenizer(reduce_len=True)
#     tokens = tweet.split(' ')
    tokens = twtk.tokenize(tweet)
    text_token = []
    for token in tokens:
        if token not in stop_words:
            text_token.append(token)
    for token in text_token:
        if token=='':
            continue
        elif token=='USER' or token=='URL': 
            stemmed_text_token.append(token)
        
        else:
            a=stem_map.get(token,0)
            if a==0:
                a=stemmer.stem(token)
                stem_map[token]=a
            stemmed_text_token.append(a)
    return ' '.join(stemmed_text_token)

In [189]:
train_file = './train_en.tsv'
train_raw_data,train_labels_TR,train_labels_AG,train_labels_HS = load_data(train_file)
train_data = [preprocess(tweet) for tweet in train_raw_data]


In [190]:
dev_file = './dev_en.tsv'
dev_raw_data,dev_labels_TR,dev_labels_AG,dev_labels_HS = load_data(dev_file)
dev_data = [preprocess(tweet) for tweet in dev_raw_data]


In [191]:
train_data[:5]

['hurray save us mani way USER USER lockthemup buildthewal enddaca boycottnfl boycottnik',
 'whi would young fight age men vast major one escap war amp cannot fight like women children elder it major refuge actual refuge econom migrant tri get europ URL',
 'USER illeg dump kid border like road kill refus unit they hope get amnesti free educ welfar illeg familesbelongtogeth countri taxpay dime it scam nodaca noamnesti sendth',
 'ny time near all white state pose array problem immigr URL URL',
 'orban brussel european leader ignor peopl want migrant URL']

In [192]:
dev_data[:5]

['i swear i get place nick time it exhaust USER rt USER made bed lie flea pit creat tourist go home refuge welcom barcelona chose migrant visitor URL',
 'i immigr trump right immigr URL give insight stori URL',
 'illegalimmigr illegalalien electoralsystem electoralcolleg i go shock peopl america not democraci america republ even defin america repres republ in true URL',
 'USER we invas issu mexican buildthatwal',
 'worker charg with sexual molest eight children immigr shelter URL']

In [193]:
X_train,y_train = train_data,train_labels_HS
X_test,y_test = dev_data,dev_labels_HS

In [194]:
# # data = np.concatenate((X_train, X_test), axis=0)
# y_test = np_utils.to_categorical(y_test)
# y_train = np_utils.to_categorical(y_train)
# classes = np.concatenate((y_train, y_test), axis=0)
y_test = np.array(y_test)
y_train = np.array(y_train)

In [195]:
MAX_NB_WORDS = 100000

In [196]:
print('loading word embeddings...')
embeddings_index = {}
def load_embeddings(fileName):
    f = codecs.open(fileName, encoding='utf-8')
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('found %s word vectors' % len(embeddings_index))


loading word embeddings...


In [197]:
#training params
batch_size = 256 
num_epochs = 30 

#model parameters
num_filters = 64 
embed_dim = 100 
weight_decay = 1e-4
MAX_NB_WORDS = 100000

In [198]:
max_seq_len = 300
print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(X_train + X_test)  #leaky
word_seq_train = tokenizer.texts_to_sequences(X_train)
word_seq_test = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

#pad sequences
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

tokenizing input data...
dictionary size:  13053


In [200]:
fileName = '../classification/glove.6B.100d.txt'
embedding_matrix = load_embeddings(fileName)

400000it [00:17, 22425.47it/s]

found 400000 word vectors


In [201]:
#embedding matrix
print('preparing embedding matrix...')
words_not_found = []
length = len(word_index) + 1
nb_words = min(MAX_NB_WORDS, length)
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))


preparing embedding matrix...
number of null word embeddings: 5293


In [204]:
num_classes = 1
print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words, embed_dim,
          weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(3))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(3))
model.add(Conv1D(num_filters, 3, activation='relu', padding='same'))
model.add(Conv1D(num_filters, 5, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

training CNN ...
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 300, 100)          1305400   
_________________________________________________________________
conv1d_69 (Conv1D)           (None, 300, 64)           44864     
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 100, 64)           0         
_________________________________________________________________
conv1d_70 (Conv1D)           (None, 100, 64)           28736     
_________________________________________________________________
max_pooling1d_36 (MaxPooling (None, 33, 64)            0         
_________________________________________________________________
conv1d_71 (Conv1D)           (None, 33, 64)            12352     
_________________________________________________________________
conv1d_72 (Conv1D)           (None, 

In [84]:
# model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_val, y_val))

# y_pred = (model.predict(x_test, batch_size=batch_size) > .5).astype(int)

In [205]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]
hist = model.fit(word_seq_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=0.1, shuffle=True, verbose=2)

Train on 8100 samples, validate on 900 samples
Epoch 1/30
 - 5s - loss: 0.6819 - accuracy: 0.5633 - val_loss: 0.6616 - val_accuracy: 0.5889
Epoch 2/30
 - 5s - loss: 0.6032 - accuracy: 0.6670 - val_loss: 0.9192 - val_accuracy: 0.3478
Epoch 3/30
 - 5s - loss: 0.5497 - accuracy: 0.7269 - val_loss: 0.8460 - val_accuracy: 0.4533
Epoch 4/30
 - 5s - loss: 0.4975 - accuracy: 0.7728 - val_loss: 0.8309 - val_accuracy: 0.5122
Epoch 5/30
 - 5s - loss: 0.4386 - accuracy: 0.8102 - val_loss: 0.7917 - val_accuracy: 0.5678
Epoch 6/30
 - 5s - loss: 0.3733 - accuracy: 0.8496 - val_loss: 0.9722 - val_accuracy: 0.5311
Epoch 7/30
 - 5s - loss: 0.2737 - accuracy: 0.8965 - val_loss: 1.1969 - val_accuracy: 0.5222
Epoch 8/30
 - 5s - loss: 0.2120 - accuracy: 0.9296 - val_loss: 1.5278 - val_accuracy: 0.5089
Epoch 9/30
 - 5s - loss: 0.1295 - accuracy: 0.9610 - val_loss: 1.5470 - val_accuracy: 0.5367
Epoch 10/30
 - 5s - loss: 0.0790 - accuracy: 0.9785 - val_loss: 2.3965 - val_accuracy: 0.5011
Epoch 11/30
 - 5s - lo

In [160]:
print("sample words not found: ", np.random.choice(words_not_found, 10))


sample words not found:  ['ducation' 'alreday' 'oitnb' 'fricken' 'khammasi' 'boyf' 'nationoflaws'
 'musicmakesadifference' 'hateposters' 'endcatchandrelease']


In [207]:
y_predict = model.predict(word_seq_test)

y_predict = np.argmax(y_predict)
y_test = np.argmax(y_test)

print("Precision\t", precision_score(y_test, y_predict, average=None))
print("Recall   \t", recall_score(y_test, y_predict, average=None))
print("F1-Score \t", f1_score(y_test, y_predict, average=None))
print("Accuracy", accuracy_score(y_test, y_predict))

TypeError: Singleton array 2 cannot be considered a valid collection.

In [187]:
y_train[:10]

array([1, 1, 1, 0, 0, 1, 0, 0, 0, 0])